In [1]:
# Criar o navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time

# Bibliotecas necessárias para envio do e-mail pela API do Gmail
from googleapiclient.discovery import build
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from google_auth_oauthlib.flow import InstalledAppFlow
import pybase64

# Tirei os avisos para rodar mais limpo, contudo pode ser necessário para avaliação futura
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

Funções de busca no Google Shopping e no Buscape

In [2]:
# Atualizar se necessário o XPATH, CLASS_NAME, etc.
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # Entrar no google
    nav.get("https://shopping.google.com/")
       
    # Tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    

    # Pesquisar o nome do produto no google
    nav.find_element(By.ID, 
                     'REsRA').send_keys(produto)
    nav.find_element(By.ID, 
                     'REsRA').send_keys(Keys.ENTER)

    # Carregar a lista de resultados da busca no Google Shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # Para cada resultado, verificar se atende a todas as condicoes
    lista_ofertas = [] # Lista com as respostas
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()

        # Verificar o nome - se contem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        
        # Verificar se o nome contem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto: # Condicao apos verificacao do nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                
                # Verificar se o preco esta dentro do range de preco
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'mnIHsc')
                    elemento_pai = elemento_link.find_elements(By.CLASS_NAME, 'shntl')
                    link = elemento_pai[0].get_attribute('href')
                    lista_ofertas.append((nome, preco, link)) # Caso verdadeiro inclui na lista de oferta
            except:
                continue
            
    return lista_ofertas
    

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # Tratar os valores que vieram da tabela
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # Entrar no buscape
    nav.get("https://www.buscape.com.br/")
    
    # Pesquisar pelo produto no Buscape
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # Carregar a lista de resultados da busca no Buscape
    time.sleep(6) # Aguardar carregar, dependendo da velocidade da internet
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'Hits_ProductCard__Bonl_')
    
    # Para cada resultado, verificar se atende a todas as condicoes
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
            nome = resultado.find_element(By.CLASS_NAME, 'Text_Text__ARJdp').text
            nome = nome.lower()

            link_class = resultado.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')
            link = link_class.get_attribute('href')
            
            # Verificar o nome - se contem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # Verificar se o nome contem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # Verificar se o preco esta dentro do range de preco
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link)) # Caso verdadeiro inclui na lista de oferta
        except:
            pass
    return lista_ofertas

In [3]:
# Abrir o navegador do Chrome
nav = webdriver.Chrome()

Contrução da nossa lista de ofertas encontradas

In [ ]:
# Importar e visualizar produtos a serem buscados na WEB (realizaremos apenas busca no GoogleShopping e Buscape)
tabela_produtos = pd.read_excel("Produtos.xlsx")
# display(tabela_produtos)

In [ ]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

# display(tabela_ofertas) # Caso queira verificar a tabela de ofertas

Exportar a base de ofertas para Excel

In [ ]:
# Exportar tabela de ofertas para o Excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)


Enviando o e-mail

In [ ]:
# Carregue as credenciais do Google API
creds = None
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.modify', 'https://www.googleapis.com/auth/gmail.compose']


# Carrega as credeciais - Roda novamente cada vez que o programa é disparado
credentials_file = 'credenciais.json' # Utilize suas próprias credenciais (https://console.cloud.google.com/apis/dashboard)
flow = InstalledAppFlow.from_client_secrets_file(
            credentials_file, SCOPES)
creds = flow.run_local_server(port=0)

# Crie um objeto de serviço para a API do Gmail
service = build('gmail', 'v1', credentials=creds)

In [ ]:
# enviar por e-mail o resultado da tabela
# verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    mensagem = MIMEMultipart()
    mensagem['From'] = 'thiagokato@gmail.com'
    mensagem['To'] = 'thiagokato@gmail.com'
    mensagem['Subject'] = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    corpo_html = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    # Adicione a parte HTML do corpo da mensagem
    parte_html = MIMEText(corpo_html, 'html')
    mensagem.attach(parte_html)

    # Converta a mensagem em uma string codificada em Base64
    mensagem_bytes = mensagem.as_bytes()
    mensagem_codificada = pybase64.b64encode(mensagem_bytes).decode('utf-8')

    # Envie o e-mail
    requisicao = {
        'raw': mensagem_codificada
    }
    service.users().messages().send(userId='me', body=requisicao).execute()

nav.quit()  